https://www.kaggle.com/mpearmain/homesite-quote-conversion/xgboost-benchmark/files

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

seed = 260681

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

y = train.QuoteConversion_Flag.values
train = train.drop(['QuoteNumber', 'QuoteConversion_Flag'], axis=1)
test = test.drop('QuoteNumber', axis=1)

# Lets play with some dates
train['Date'] = pd.to_datetime(pd.Series(train['Original_Quote_Date']))
train = train.drop('Original_Quote_Date', axis=1)

test['Date'] = pd.to_datetime(pd.Series(test['Original_Quote_Date']))
test = test.drop('Original_Quote_Date', axis=1)

train['Year'] = train['Date'].apply(lambda x: int(str(x)[:4]))
train['Month'] = train['Date'].apply(lambda x: int(str(x)[5:7]))
train['weekday'] = train['Date'].dt.dayofweek


test['Year'] = test['Date'].apply(lambda x: int(str(x)[:4]))
test['Month'] = test['Date'].apply(lambda x: int(str(x)[5:7]))
test['weekday'] = test['Date'].dt.dayofweek

train = train.drop('Date', axis=1)
test = test.drop('Date', axis=1)

train = train.fillna(-1)
test = test.fillna(-1)

In [4]:
train.columns

Index([u'Field6', u'Field7', u'Field8', u'Field9', u'Field10', u'Field11',
       u'Field12', u'CoverageField1A', u'CoverageField1B', u'CoverageField2A',
       ...
       u'GeographicField60B', u'GeographicField61A', u'GeographicField61B',
       u'GeographicField62A', u'GeographicField62B', u'GeographicField63',
       u'GeographicField64', u'Year', u'Month', u'weekday'],
      dtype='object', length=299)

In [5]:
for f in train.columns:
    if train[f].dtype=='object':
        print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train[f].values) + list(test[f].values))
        train[f] = lbl.transform(list(train[f].values))
        test[f] = lbl.transform(list(test[f].values))

Field6
Field10
Field12
CoverageField8
CoverageField9
SalesField7
PersonalField7
PersonalField16
PersonalField17
PersonalField18
PersonalField19
PropertyField3
PropertyField4
PropertyField5
PropertyField7
PropertyField14
PropertyField28
PropertyField30
PropertyField31
PropertyField32
PropertyField33
PropertyField34
PropertyField36
PropertyField37
PropertyField38
GeographicField63
GeographicField64


In [7]:
import numpy as np
from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import load_iris, load_digits, load_boston
import pickle
import xgboost as xgb

clf = xgb.XGBClassifier(n_estimators=25,
                        nthread=-1,
                        max_depth=10,
                        learning_rate=0.025,
                        silent=True,
                        subsample=0.8,
                        colsample_bytree=0.8)
                        
xgb_model = clf.fit(train, y, eval_metric="auc")

OSError: dlopen(/Users/howard/anaconda/envs/dato-env/lib/python2.7/site-packages/libxgboostwrapper.so, 6): Library not loaded: @rpath/./libgomp.1.dylib
  Referenced from: /Users/howard/anaconda/envs/dato-env/lib/python2.7/site-packages/libxgboostwrapper.so
  Reason: image not found

In [1]:
preds = clf.predict_proba(test)[:,1]
sample = pd.read_csv('../input/sample_submission.csv')
sample.QuoteConversion_Flag = preds
sample.to_csv('xgb_benchmark.csv', index=False)

NameError: name 'clf' is not defined